import library

In [15]:
import torch
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification
from scipy.io.wavfile import read

config

In [16]:
audio_path = ["../data/1-17-2017/0.wav", "../data/1-17-2017/1.wav", "../data/1-17-2017/2.wav", "../data/1-17-2017/3.wav", "../data/1-17-2017/4.wav", "../data/1-17-2017/5.wav"]

modelw = Wav2Vec2ForSequenceClassification.from_pretrained('alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech')
processor = Wav2Vec2FeatureExtractor.from_pretrained('alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label2id = {
    "female": 0,
    "male": 1
}

id2label = {
    0: "female",
    1: "male"
}

num_labels = 2

Some weights of the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

inference

In [17]:
results_dict = {}

def store_results(prob, file_name):
  prob_list = prob.tolist()[0]
  if prob_list[1] > prob_list[0]:
    gender = "Female"
    probability = prob_list[1]
  else:
    gender = "Male"
    probability = prob_list[0]
  results_dict[file_name] = {'pred': gender, 'prob': probability}

for path in audio_path:
  output = read(path)
  waveform = np.array(output[1],dtype=float)
  sound_array = np.array(waveform)
  input_values = processor(sound_array, sampling_rate = 16000, padding='longest', return_tensors='pt').input_values
  with torch.no_grad():
    result = modelw(input_values).logits
    prob = torch.nn.functional.softmax(result, dim=1)
    store_results(prob, path)

: 

results

In [ ]:
results_dict

{'../data/1/1.wav': {'pred': 'Female', 'prob': 0.9986407160758972}}